<a href="https://colab.research.google.com/github/Akanksha1212/BERT-Classification/blob/master/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Environment setup 

In [39]:
import pandas as pd
import numpy as np
import io

Import csv 

In [40]:
from google.colab import files
uploaded = files.upload()

Saving DF1.csv to DF1 (5).csv


In [41]:
data = pd.read_csv(io.StringIO(uploaded['DF1.csv'].decode('utf-8')))

In [42]:
data.head()

,Title,Category,Post Text
0,Testing requirements in toys and games category,Amazon Custom,My product is domino sets for adults. I am the...
1,Food and Drug Administration,Amazon Custom,"Hello, I am on my way to becoming an Amazon de..."
2,Images Storage for mass upload,Amazon Custom,When uploading products with the spreadsheet w...
3,Prepaid Return Label program email clarification,Amazon Custom,You may have received an email from Amazon inf...
4,Pulling Amazon Custom Personalizatoin Data fro...,Amazon Custom,The problem: Currently Amazon gives you links ...


Make subdata and encode Category 

In [43]:
subdata = data[["Title", "Category", "Post Text"]]
subdata.head()

,Title,Category,Post Text
0,Testing requirements in toys and games category,Amazon Custom,My product is domino sets for adults. I am the...
1,Food and Drug Administration,Amazon Custom,"Hello, I am on my way to becoming an Amazon de..."
2,Images Storage for mass upload,Amazon Custom,When uploading products with the spreadsheet w...
3,Prepaid Return Label program email clarification,Amazon Custom,You may have received an email from Amazon inf...
4,Pulling Amazon Custom Personalizatoin Data fro...,Amazon Custom,The problem: Currently Amazon gives you links ...


In [45]:
subdata['Category'] = subdata['Category'].astype('category')
subdata['Label'] = subdata['Category'].cat.codes.astype('int32')
subdata.sample(5)

,Title,Category,Post Text,Label
1104,Amazon not accepting my POA and appeals to rea...,Account Health,Hey all. Heres the original email I received f...,0
953,Different Types of Inauthentic Complaints,Account Health,I do not understand the different types of Ina...,0
401,"FBA CLT2 Warehouse, Goods delivered on Friday ...",Fulfillment By Amazon,"my FBA warehouse is basically CLT2 Charlotte, ...",2
1418,Potential Trademark Misuse,Account Health,"Hello, Looking for some advice\nToday we recei...",0
975,Account Deactivated (New Seller) (Help),Account Health,"Hi everybody,\nIm a new seller on Amazon with ...",0


Transformer installation from HuggingFace (reload kernel after running for the first time) 

In [46]:
pip install transformers

In [47]:
# More setup  
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

from transformers import DistilBertTokenizer

#### DistilBERT 

In [48]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

# For BERT: 
# model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

#### Tokenize 

In [49]:
tokenized = subdata["Post Text"].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,truncation=True, max_length=100)))

In [50]:
tokenized

0       [101, 2026, 4031, 2003, 23968, 4520, 2005, 600...
1       [101, 7592, 1010, 1045, 2572, 2006, 2026, 2126...
2       [101, 2043, 2039, 18570, 3688, 2007, 1996, 208...
3       [101, 2017, 2089, 2031, 2363, 2019, 10373, 201...
4       [101, 1996, 3291, 1024, 2747, 9733, 3957, 2017...
                              ...                        
1810    [101, 2322, 1003, 1997, 1996, 11372, 2024, 200...
1811    [101, 7592, 2372, 1010, 1045, 2572, 17942, 200...
1812    [101, 2061, 1045, 2572, 1037, 2047, 14939, 199...
1813    [101, 7632, 1010, 10047, 1037, 2047, 14939, 20...
1814    [101, 2026, 4070, 2001, 26709, 6593, 21967, 23...
Name: Post Text, Length: 1815, dtype: object

#### Padding 

In [51]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i) # Find longest 

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values]) # Adding 0's 

In [52]:
np.array(padded).shape

(1815, 100)

#### Masking 

In [53]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1815, 100)

#### Run through BERT 

In [54]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [55]:
last_hidden_states

(tensor([[[ 1.5445e-01, -7.8006e-02,  1.9231e-01,  ..., -7.1839e-02,
            3.2483e-01,  4.1357e-01],
          [ 1.9706e-01, -2.0696e-01, -5.1620e-02,  ...,  3.0763e-02,
            5.7762e-01,  4.1447e-01],
          [-1.6024e-02, -2.9119e-01,  3.4074e-01,  ..., -1.3788e-01,
           -1.2856e-01, -1.0083e-01],
          ...,
          [ 4.4683e-01, -8.6159e-02, -1.5706e-02,  ...,  1.6105e-02,
            1.9443e-01,  4.6065e-01],
          [ 2.2944e-01,  6.6831e-02,  1.7589e-01,  ..., -1.6872e-01,
            2.3243e-02,  1.9865e-01],
          [-3.5367e-01,  1.5651e-01,  9.0538e-01,  ...,  2.8317e-01,
            2.3495e-01, -7.3744e-02]],
 
         [[ 1.7135e-01, -1.0024e-01,  8.7762e-02,  ..., -2.9089e-02,
            3.7860e-01,  3.9617e-01],
          [ 9.8955e-02,  1.2467e-01,  5.6948e-01,  ...,  1.9390e-01,
            4.9541e-01,  6.9128e-02],
          [-5.6559e-01,  4.0507e-01,  6.6953e-01,  ..., -1.8603e-02,
            2.6326e-01,  1.6061e-01],
          ...,
    

#### Classification 

Get features  

In [56]:
features = last_hidden_states[0][:,0,:].numpy()

In [57]:
labels = subdata["Label"] 

Train Test Split 

In [58]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

Logistic Regression (Can try Decsion Tree, Random Forest, etc.) 

In [59]:
log = LogisticRegression()
log.fit(train_features, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Accuracy 

In [60]:
log.score(test_features, test_labels)

0.8325991189427313